# ARIMA Modeling of GDP (Univariate Forecasting)

## Purpose
This notebook builds a **univariate** ARIMA model for India’s GDP (current USD, billions) and produces out-of-sample forecasts.

The workflow follows the Box–Jenkins approach:
1. Load and structure the GDP time series
2. Visual inspection
3. Stationarity testing (ADF)
4. Transformations to achieve stationarity (log / differencing)
5. Identification using ACF/PACF
6. Model selection and estimation (ARIMA)
7. Validation with a simple holdout split
8. Residual diagnostics and forecasting

> Note: This notebook focuses only on GDP’s internal dynamics (univariate). Multivariate relationships (e.g., VAR / Granger) are handled in the separate notebook.


## 0. Setup
We import the required libraries and define a small set of helper functions used throughout the analysis.

In [ ]:
# Core
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Stats / time-series
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.stats.diagnostic import acorr_ljungbox

from sklearn.metrics import mean_squared_error


## 1. Load data and create a GDP time series

The dataset is stored in the repository under `data/`.  
We convert the `Years` column into a proper datetime index and isolate GDP.

In [ ]:
DATA_PATH = Path("..") / "data" / "gdp data.csv"

df = pd.read_csv(DATA_PATH)

# Create a datetime index (annual frequency)
df["Years"] = pd.to_datetime(df["Years"].astype(int).astype(str), format="%Y")
df = df.sort_values("Years").set_index("Years")

gdp_col = "GDP (current USD in billions)"
gdp = df[gdp_col].astype(float)

df[[gdp_col]].head()

## 2. Visual inspection

GDP typically exhibits trend growth over long horizons.  
A visible trend is a common indicator of non-stationarity.

In [ ]:
plt.figure(figsize=(12,5))
gdp.plot(marker=".")
plt.title("India GDP (current USD, billions)")
plt.ylabel("USD (billions)")
plt.grid(alpha=0.3)
plt.show()

## 3. Stationarity testing (ADF)

ARIMA requires a stationary series. We use the Augmented Dickey–Fuller test:

- **H0:** unit root (non-stationary)  
- **H1:** stationary

If p-value > 0.05, we typically difference (and/or transform) the series.

In [ ]:
def adf_report(series: pd.Series, name: str = "series"):
    series = series.dropna()
    stat, pvalue, lags, nobs, crit, _ = adfuller(series, autolag="AIC")
    out = {
        "series": name,
        "adf_stat": stat,
        "p_value": pvalue,
        "lags": lags,
        "n_obs": nobs,
        "crit_1%": crit["1%"],
        "crit_5%": crit["5%"],
        "crit_10%": crit["10%"],
    }
    return pd.Series(out)

adf_report(gdp, "GDP (level)")

## 4. Transformations to achieve stationarity

We test common transformations:
- **log(GDP)** to stabilize variance
- **first difference of log(GDP)** to remove trend (typical for macro series)

We choose a transformation that yields stationarity and is economically interpretable.

In [ ]:
gdp_log = np.log(gdp).replace([np.inf, -np.inf], np.nan).dropna()
gdp_log_diff = gdp_log.diff().dropna()

pd.concat(
    [adf_report(gdp_log, "log(GDP)"),
     adf_report(gdp_log_diff, "diff(log(GDP))")],
    axis=1
)

## 5. Identification via ACF/PACF

After stationarity is achieved, we use ACF/PACF to guide AR (p) and MA (q) orders.

- PACF “cutoff” suggests AR order  
- ACF “cutoff” suggests MA order  
- Slow decay often suggests mixed ARMA components

In [ ]:
fig = plt.figure(figsize=(12,8))

ax1 = fig.add_subplot(211)
sm.graphics.tsa.plot_acf(gdp_log_diff, lags=12, ax=ax1)
ax1.set_title("ACF: diff(log(GDP))")

ax2 = fig.add_subplot(212)
sm.graphics.tsa.plot_pacf(gdp_log_diff, lags=12, ax=ax2, method="ywm")
ax2.set_title("PACF: diff(log(GDP))")

plt.tight_layout()
plt.show()

## 6. Train/test split

For a simple validation, we hold out the final 10 observations as a test set.  
This is not a full forecasting competition setup, but it provides a sanity-check on out-of-sample performance.

In [ ]:
test_size = 10
train = gdp.iloc[:-test_size]
test = gdp.iloc[-test_size:]

train.index.min(), train.index.max(), test.index.min(), test.index.max(), (train.shape, test.shape)

## 7. Model selection (parsimonious grid)

We evaluate a small grid of ARIMA(p, d, q) models on the **level GDP** series using a holdout split.

Notes:
- GDP in levels typically needs differencing (d >= 1).  
- We keep the grid small for clarity and reproducibility.

We select the model with the lowest RMSE on the test set.

In [ ]:
from itertools import product

# Small, transparent grid (you can expand if needed)
p_vals = [0, 1, 2, 3]
d_vals = [1, 2]
q_vals = [0, 1, 2, 3]

candidates = list(product(p_vals, d_vals, q_vals))

results = []
for order in candidates:
    try:
        model = ARIMA(train, order=order).fit()
        pred = model.predict(start=test.index[0], end=test.index[-1])
        rmse = float(np.sqrt(mean_squared_error(test, pred)))
        results.append((order, rmse, model.aic, model.bic))
    except Exception:
        continue

grid = pd.DataFrame(results, columns=["order", "rmse", "aic", "bic"]).sort_values("rmse").reset_index(drop=True)
grid.head(10)

## 8. Fit the selected model and evaluate

We fit the best-performing (lowest RMSE) specification on the training set and evaluate predictions on the test period.

In [ ]:
best_order = tuple(grid.loc[0, "order"])
best_order

### Fit and predict (holdout)

We compare fitted predictions against the held-out observations.

In [ ]:
best_model = ARIMA(train, order=best_order).fit()

pred_test = best_model.predict(start=test.index[0], end=test.index[-1])
rmse = float(np.sqrt(mean_squared_error(test, pred_test)))

rmse

### Plot: train, test, and predictions

In [ ]:
plt.figure(figsize=(12,5))
train.plot(label="train")
test.plot(label="test")
pred_test.plot(label=f"pred (ARIMA{best_order})")
plt.title("Holdout evaluation")
plt.ylabel("GDP (USD, billions)")
plt.grid(alpha=0.3)
plt.legend()
plt.show()

## 9. Residual diagnostics

A well-specified ARIMA model should leave residuals that resemble white noise.

We check:
- Residual time plot
- Residual ACF
- Ljung–Box test (null: no autocorrelation up to given lags)

In [ ]:
resid = best_model.resid.dropna()

plt.figure(figsize=(12,4))
resid.plot()
plt.title("Residuals (train fit)")
plt.grid(alpha=0.3)
plt.show()

plt.figure(figsize=(12,4))
sm.graphics.tsa.plot_acf(resid, lags=12, ax=plt.gca())
plt.title("ACF of residuals")
plt.tight_layout()
plt.show()

lb = acorr_ljungbox(resid, lags=[10], return_df=True)
lb

## 10. Refit on full sample and forecast

After selecting a reasonable specification, we refit the model on the full GDP series and produce forecasts.

Forecasts are accompanied by 95% confidence intervals.

In [ ]:
final_model = ARIMA(gdp, order=best_order).fit()

horizon = 16  # number of future periods
fc = final_model.get_forecast(steps=horizon)
fc_mean = fc.predicted_mean
fc_ci = fc.conf_int()

forecast_table = pd.DataFrame({
    "GDP_actual": gdp,
    "GDP_forecast": fc_mean,
    "lower_95": fc_ci.iloc[:, 0],
    "upper_95": fc_ci.iloc[:, 1],
})

forecast_table.tail(10)

### Plot: historical GDP and forecasts

In [ ]:
plt.figure(figsize=(12,5))
gdp.plot(label="historical")
fc_mean.plot(label="forecast")
plt.fill_between(fc_ci.index, fc_ci.iloc[:,0], fc_ci.iloc[:,1], alpha=0.2)
plt.title(f"GDP forecast using ARIMA{best_order}")
plt.ylabel("GDP (USD, billions)")
plt.grid(alpha=0.3)
plt.legend()
plt.show()

## 11. Save outputs

We export the forecast table to the repository `data/` folder for easy reuse (e.g., in a report or README).

In [ ]:
OUT_PATH = Path("..") / "data" / "results.csv"
forecast_table.to_csv(OUT_PATH, index=True)
OUT_PATH

# Conclusion

- GDP in levels shows non-stationarity consistent with long-run growth.
- After selecting an ARIMA specification with a transparent holdout evaluation, the fitted model provides a baseline GDP forecast.
- Residual diagnostics (ACF + Ljung–Box) help validate whether remaining autocorrelation is minimal.

This notebook is intended as a clean, reproducible baseline. For richer macroeconomic inference, complement this with multivariate models (e.g., VAR / Granger causality).